In [114]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

# Importing Image class from PIL module 
from PIL import Image
import boto3

#returns the different labels of objects that are found in the photo
def detect_labels(photo, bucket):

    client=boto3.client('rekognition')

    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
        MaxLabels=15)

    return (response['Labels'])


#checks if a person is detected in the photo and returns the objectbox values
def getPersonObjectBox(labels):

    for label in labels:
        if label['Name'] == "Person":
            for instance in label['Instances']:
                personObjectBox = {
                    "Type": "Person",
                    "width": instance['BoundingBox']['Width'],
                    "height": instance['BoundingBox']['Height'],
                    "top": instance['BoundingBox']['Top'],
                    "left": instance['BoundingBox']['Left'],
                }
                return personObjectBox
            else:
                return False

def cropImage(left, upper, right, lower, index, path):

    #change path to use \ for windows compatibility

    path = path.replace('/', '\\')

    # Opens a image in RGB mode 
    im = Image.open(f'C:\ANPR_FOTOS\Alarm Picture\{path}')

    im_crop = im.crop((left, upper, right, lower))

    im_crop.save(f'C:\ANPR_FOTOS\Alarm Picture\Cropped\cropped{index}.jpg', 'JPEG')


def calculateCoordinates(imageWidth, imageHeight, person):

    print(f'HIERE: {person}')

    #calculate the coördinates of the person objectBox 

    #calculate top/left (x,y) coördinate

    x1 = imageWidth * person['left']
    y1 = imageHeight * person['top']

    #calculate bottom/right (x,y) coördinate

    x2 = x1 + imageWidth * person['width']
    y2 = y1 + imageHeight * person['height']

    #add margins to get the windshield of the car left - 40%, right + 20%, top - 15% and bottom + 15%
    
    x1 = x1 - x1 * 0.4
    y1 = y1 - y1 * 0.15

    x2 = x2 + x2 * 0.2
    y2 = y2 + y2 * 0.15

    coordinates = {
        "x1": x1,
        "y1": y1,
        "x2": x2,
        "y2": y2,
    }

    return coordinates




def main():
    # photo='Fotos-7-12-2020/20000101005741860_1_1SHP373_1_8.jpg'
    bucket='ai-essentials-anpr'

    #loop over a list of all images in the s3 bucket and crop all windshields out of the photographs

    conn = client('s3')

    #enumerate is used to keep track of the index in the loop
    for index, key in enumerate(conn.list_objects(Bucket='ai-essentials-anpr')['Contents']):
        #key['Key'] is the name of a photo in the s3 bucket
        labels=detect_labels(key['Key'], bucket)

        person = getPersonObjectBox(labels)

        if(person):
            #image dimensions
            imageWidth = 1920
            imageHeight = 1144

            coordinates = calculateCoordinates(imageWidth, imageHeight, person)

            print(coordinates['x1'], coordinates['y1'])
            print(coordinates['x2'], coordinates['y2'])

            cropImage(coordinates['x1'], coordinates['y1'], coordinates['x2'], coordinates['y2'], index, key['Key'])
        else:
            print('No person detected')
        


if __name__ == "__main__":
    main()






HIERE: {'Type': 'Person', 'width': 0.06595519185066223, 'height': 0.12109780311584473, 'top': 0.1640293151140213, 'left': 0.5231375694274902}
602.6544799804688 159.5021060168743
1357.2697219848633 375.11323674321176
HIERE: {'Type': 'Person', 'width': 0.0904439315199852, 'height': 0.13863599300384521, 'top': 0.129844531416893, 'left': 0.522415280342102}
601.8224029541016 126.26082234978676
1412.027624130249 353.2129779279232
HIERE: {'Type': 'Person', 'width': 0.07100117951631546, 'height': 0.1280696839094162, 'top': 0.16073989868164062, 'left': 0.5073753595352173}
584.4964141845703 156.30347747802733
1332.5795459747314 379.95788685679435
HIERE: {'Type': 'Person', 'width': 0.10834947973489761, 'height': 0.15524432063102722, 'top': 0.13076844811439514, 'left': 0.5170868039131165}
595.6839981079102 127.15923894643784
1441.0051975250244 376.2783985614777
HIERE: {'Type': 'Person', 'width': 0.11250686645507812, 'height': 0.15746502578258514, 'top': 0.12953482568264008, 'left': 0.5191332697868

In [104]:
from boto3 import client

conn = client('s3')
for key in conn.list_objects(Bucket='ai-essentials-anpr')['Contents']:
    print(key['Key'])

Fotos-7-12-2020/20000101002551180_1_1SHP373_1_12.jpg
Fotos-7-12-2020/20000101003421656_1_1SHP373_1_5.jpg
Fotos-7-12-2020/20000101003625593_1_1SHP373_1_1.jpg
Fotos-7-12-2020/20000101005122011_1_1SHP373_1_2.jpg
Fotos-7-12-2020/20000101005448076_1_1SHP373_1_10.jpg
Fotos-7-12-2020/20000101005536929_1_1SHP373_1_12.jpg
Fotos-7-12-2020/20000101005538933_1_1SHP373_1_13.jpg
Fotos-7-12-2020/20000101005659489_1_1SHP373_1_1.jpg
Fotos-7-12-2020/20000101005710573_1_1SHP373_1_2.jpg
Fotos-7-12-2020/20000101005721333_1_1SHP373_1_3.jpg
Fotos-7-12-2020/20000101005725564_1_1SHP373_1_4.jpg
Fotos-7-12-2020/20000101005728614_1_1SHP373_1_5.jpg
Fotos-7-12-2020/20000101005736481_1_1SHP373_1_6.jpg
Fotos-7-12-2020/20000101005739004_1_1SHP373_1_7.jpg
Fotos-7-12-2020/20000101005741860_1_1SHP373_1_8.jpg
Fotos-7-12-2020/20000101005748493_1_1SHP373_1_9.jpg
Fotos-7-12-2020/20000101005750436_1_1SHP373_1_10.jpg
Fotos-7-12-2020/20000101005752424_1_1SHP373_1_11.jpg
Fotos-7-12-2020/20000101005753560_1_1SHP373_1_12.jpg
Fotos